### TP5-6 Julie Chapdelaine et Cleo Daguin


Ce TP va nous permettre de comprendre les réseaux neuronaux convolutifs et de faire du traitement d'image et de la classification

#### **I) Understanding convolutional networks for computer vision**
Nous allons dans un premier temps étudier un réseau convolution appliqué à la base de donnée mnist (vue dans le TP 1-2). Source : https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


Normalisation de la taille des images (img-rows*img-cols)

In [0]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

Transformation des images en matrice de float32 représentant chaque pixel

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


Transformation des sorties attendues pour etre un vecteur de 10 elements binaires dont un seul est à 1 (comme dans les tps précédents pour une sortie en catégories)

In [0]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

**Création du model**


*   Une ou plusieurs couches de convolutions qui forment un prétraitement
*   Un pooling qui rassemble les résultats du prétraitement
*   Un dropout pour éviter l'overfitting
*   Un flatten pour revenir à 1 dimension
*   Un dense pour le dur du traitement
*   Un second dropout
*   Un second dense avec une sortie de taille 10 pour les 10 catégories possibles



In [0]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/12





60000/60000 [==============================] - 152s 3ms/step - loss: 0.2625 - acc: 0.9202 - val_loss: 0.0547 - val_acc: 0.9829
Epoch 2/12
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0887 - acc: 0.9739 - val_loss: 0.0384 - val_acc: 0.9877
Epoch 3/12
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0637 - acc: 0.9812 - val_loss: 0.0352 - val_acc: 0.9871
Epoch 4/12
60000/60000 [==============================] - 151s 3ms/step - loss: 0.0549 - acc: 0.9835 - val_loss: 0.0345 - val_acc: 0.9893
Epoch 5/12
60000/60000 [==============================] - 152s 3ms/step - loss: 0.0463 - acc: 0.9854 - val_loss: 0.0299 - val_acc: 0.9897
Epoch 6/12
60000/60000 [============

On peut observer qu'avec seulement 12 epochs, l'accuracy est déjà à 99,19 % et que la loss et la val-loss sont très basses ce qui est recherché.

In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)              

explication calculs et paramètres 
*   320 => (3 * 3 + 1) * 32  première couche
*   18496 => (3 * 3 * 32 + 1) * 64  deuxième couche
*   1 179 776 => (12 * 12 * 64 + 1) * 128  premiere couche dense
*   1 290 => (128 + 1) * 10  deuxième couche dense

*   maxpooling a un stride de 2 => perte de résolution de 24 * 24 à 12 * 12
*   padding non géré ce qui explique le passage de 26 * 26 à 24 * 24 entre la 1ere et 2eme couche

Enlevons le maxPooling pour voir les conséquences.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 231s 4ms/step - loss: 0.2495 - acc: 0.9241 - val_loss: 0.0592 - val_acc: 0.9810
Epoch 2/12
60000/60000 [==============================] - 232s 4ms/step - loss: 0.0870 - acc: 0.9742 - val_loss: 0.0395 - val_acc: 0.9857
Epoch 3/12
60000/60000 [==============================] - 231s 4ms/step - loss: 0.0635 - acc: 0.9810 - val_loss: 0.0499 - val_acc: 0.9839
Epoch 4/12
60000/60000 [==============================] - 232s 4ms/step - loss: 0.0509 - acc: 0.9848 - val_loss: 0.0316 - val_acc: 0.9895
Epoch 5/12
60000/60000 [==============================] - 231s 4ms/step - loss: 0.0425 - acc: 0.9871 - val_loss: 0.0324 - val_acc: 0.9897
Epoch 6/12
60000/60000 [==============================] - 230s 4ms/step - loss: 0.0375 - acc: 0.9883 - val_loss: 0.0317 - val_acc: 0.9889
Epoch 7/12
60000/60000 [==============================] - 230s 4ms/step - loss: 0.0310 - acc: 0.9900 - val_loss: 0.0289 - 

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
dropout_3 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 36864)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               4718720   
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)               

En enlevant le maxPooling on peut voir qu'il y a environ 4 fois plus de paramères dans le réseau